<a href="https://colab.research.google.com/github/shivammahajan03/playlist_analysis/blob/main/my_playlist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Steps to understand working of SQLite3 using python

1. **Basic SQL**
Setup of a sample database
2. **Build a small dataset with popular musisc, artists, genres**
3. **Build a small recommendation engine based on the user's tastes, likes and dislikes**

In [ ]:
!pip install pysqlite3
from pysqlite3 import dbapi2 as sqlite3
print(sqlite3.sqlite_version)
import pandas as pd
from IPython.display import display, HTML


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.7/40.7 kB 929.0 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pysqlite3: filename=pysqlite3-0.5.2-cp310-cp310-linux_x86_64.whl size=151112 sha256=8e5a3aed2b5696159e8cce487fba33cad718828af74037d2ac834b663341db96
  Stored in directory: /root/.cache/pip/wheels/91/91/77/1912a316aad35d0edcb42ee1aa9b7bc07d7190a5798d378805
Successfully built pysqlite3
3.37.2


In [ ]:
def printSqlResults(caption,tblName):
  try:
    df = pd.DataFrame(caption.fetchall(), columns=[i[0] for i in caption.description])
    display(HTML("<b><font color=Green>"+tblName+"</font></b>" +df.to_html(index=False)))
  except:
    pass

def runSql(caption, query):
  conn = sqlite3.connect(dbname)
  cursor = conn.cursor()
  cursor.execute(query)
  printSqlResults(cursor,caption)
  conn.close()

In [ ]:
dbname = "my_music.db"

conn = sqlite3.connect(dbname)
cursor = conn.cursor()

cursor.execute("""
CREATE TABLE IF NOT EXISTS Users (
  user_id INTEGER PRIMARY KEY,
  name VARCHAR(100) NOT NULL,
  email VARCHAR(100) NOT NULL UNIQUE
);
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS Songs (
  song_id INTEGER PRIMARY KEY,
  title VARCHAR(100) NOT NULL,
  artist VARCHAR(100) NOT NULL,
  genre VARCHAR(100)
);
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS Listens (
  listen_id INTEGER PRIMARY KEY,
  user_id INTEGER NOT NULL,
  song_id INTEGER NOT NULL,
  rating FLOAT,
  listen_time TIMESTAMP,
  FOREIGN KEY (user_id) REFERENCES Users(user_id),
  FOREIGN KEY (song_id) REFERENCES Songs(song_id)
);
""")


cursor.execute("""
CREATE TABLE IF NOT EXISTS Recommendations (
  user_id INTEGER NOT NULL,
  song_id INTEGER NOT NULL,
  recommendation_id NOT NULL PRIMARY KEY,
  recommendation_time TIMESTAMP,
  FOREIGN KEY (user_id) REFERENCES Users(user_id),
  FOREIGN KEY (song_id) REFERENCES Songs(song_id)
);
""")

conn.commit()
conn.close()

In [ ]:
conn = sqlite3.connect(dbname)
sqlite3.enable_callback_tracebacks(True)
cursor = conn.cursor()
cursor.execute("delete from Songs")
cursor.execute("delete from Users")
cursor.execute("delete from Listens")
cursor.execute("delete from Recommendations")

cursor.execute("""
INSERT INTO Users (user_id, name, email)
VALUES
      (1, 'Mickey', 'mickey@example.com'),
      (2, 'Minnie', 'minnie@example.com'),
      (3, 'Daffy', 'daffy@example.com'),
      (4, 'Pluto', 'pluto@example.com');
""")

cursor.execute("""
INSERT INTO Songs (song_id, title, artist, genre)
VALUES
      (1, 'Evermore', 'Taylor Swift', 'Pop'),
      (2, 'Bad Blood', 'Taylor Swift', 'Rock'),
      (3, 'Willow', 'Taylor Swift', 'Pop'),
      (4, 'Shape of you', 'Ed Sheeran', 'Rock'),
      (5, 'Photograph', 'Ed Sheeran', 'Rock'),
      (6, 'Shivers', 'Ed Sheeran', 'Pop'),
      (7, 'Hey Jude', 'Beatles', 'Classic'),
      (8, 'Yellow Submarine', 'Beatles', 'Classic'),
      (9, 'Hey Jude', 'Beatles', 'Classic'),
      (10, 'DJ Mix', 'DJ', NULL);
""")


cursor.execute("""
INSERT INTO Listens (listen_id, user_id, song_id, rating)
VALUES
      (1, 1, 1, 4.5),
      (2, 1, 2, 4.2),
      (3, 1, 6, 3.9),
      (4, 2, 2, 4.7),
      (5, 2, 7, 4.6),
      (6, 2, 8, 3.9),
      (7, 3, 2, 2.9),
      (8, 3, 6, 4.9);
""")

conn.commit()
conn.close()

In [ ]:
runSql('Users', "select * from Users")
runSql('Songs', "select * from Songs")
runSql('Listens', "select * from Listens")

user_id,name,email
1,Mickey,mickey@example.com
2,Minnie,minnie@example.com
3,Daffy,daffy@example.com
4,Pluto,pluto@example.com


song_id,title,artist,genre
1,Evermore,Taylor Swift,Pop
2,Bad Blood,Taylor Swift,Rock
3,Willow,Taylor Swift,Pop
4,Shape of you,Ed Sheeran,Rock
5,Photograph,Ed Sheeran,Rock
6,Shivers,Ed Sheeran,Pop
7,Hey Jude,Beatles,Classic
8,Yellow Submarine,Beatles,Classic
9,Hey Jude,Beatles,Classic
10,DJ Mix,DJ,None


listen_id,user_id,song_id,rating,listen_time
1,1,1,4.5,None
2,1,2,4.2,None
3,1,6,3.9,None
4,2,2,4.7,None
5,2,7,4.6,None
6,2,8,3.9,None
7,3,2,2.9,None
8,3,6,4.9,None


In [ ]:
"""
Find the titles and artists in the "Classic" genre
"""

qry_classic="""
SELECT Songs.title, Songs.artist
FROM Songs
WHERE Songs.genre='Classic'
"""
runSql('Classic Songs', qry_classic)

title,artist
Hey Jude,Beatles
Yellow Submarine,Beatles
Hey Jude,Beatles


Write query to do the following:
1. List all genres
2. List unique genres
3. Songs of Taylor Swift in different genres
4. Find count of songs in each genres
5. Count of songs in each genre artists wise
6. Popular songs by the number of listens
7. Taylor Swift's songs listned to by Mickey
8. Count ratings for Beatle's songs
9. Titles and artists of songs that have been listed to by user_id=1
10. Titles and artists of songs with >= 1 listen recorded in the Listens table.
11.  Retrieve songs that have not been listened to by user with ID 1
12. Titles and artists of songs from both Pop and Rock genres
13. Artists of songs from both Pop and Rock genres
14. Titles and artists of songs from both Pop and Rock genres, including duplicates
15. Artists with songs that belong to both Pop and Rock genres
16. Artists with songs in Rock genre, but none in the Pop genre

In [ ]:
## 1. List all genres

query_1 = """
SELECT genre
FROM Songs
"""
runSql('All genre', query_1)

genre
Pop
Rock
Pop
Rock
Rock
Pop
Classic
Classic
Classic
None


In [ ]:
## 2. List all unique genres

query_2 = """
SELECT distinct genre
FROM Songs
"""
runSql('Unique genre', query_2)

genre
Pop
Rock
Classic
None


In [ ]:
## 3. Songs of Taylor Swift in different genres

query_3 = """
SELECT title, genre
FROM Songs
where artist = 'Taylor Swift'
"""
runSql('Genres of taylor Swift', query_3)

title,genre
Evermore,Pop
Bad Blood,Rock
Willow,Pop


In [ ]:
## 4. Find count of songs in each genres

query_4 = """
SELECT genre, count(genre) as count_genre
FROM Songs
group by genre
"""
runSql('Songs per genre', query_4)

genre,count_genre
None,0
Classic,3
Pop,3
Rock,3


In [ ]:
## 5. Count of songs in each genre artists wise

query_5 = """
SELECT artist, genre, count(genre) as count_genre
FROM Songs
group by artist, genre
"""
runSql('Songs per artist per genre', query_5)

artist,genre,count_genre
Beatles,Classic,3
DJ,None,0
Ed Sheeran,Pop,1
Ed Sheeran,Rock,2
Taylor Swift,Pop,2
Taylor Swift,Rock,1


In [ ]:
## 6. Popular songs by the number of listens

query_6 = """
SELECT title, count(listens.song_id) as count_song
FROM listens
join songs on songs.song_id = listens.song_id
group by listens.song_id
order by count(listens.song_id) desc
"""
runSql('Popular Songs', query_6)

title,count_song
Bad Blood,3
Shivers,2
Yellow Submarine,1
Hey Jude,1
Evermore,1


In [ ]:
## 7. Taylor Swift's songs listned to by Mickey

query_7 = """
SELECT title
from songs
join listens on songs.song_id = listens.song_id
join users on users.user_id = listens.user_id
where name = 'Mickey' and artist = 'Taylor Swift'
"""
runSql('Mickey Taylor Songs', query_7)

title
Evermore
Bad Blood


In [ ]:
## 8. Count ratings for Beatle's songs

query_8 = """
SELECT title, count(rating) as count_rating
from listens
join songs on songs.song_id = listens.song_id
where artist = 'Beatles'
group by rating
"""
runSql('Count Rating Beatles', query_8)

title,count_rating
Yellow Submarine,1
Hey Jude,1


In [ ]:
## 9. Titles and artists of songs that have been listened to by user_id = 1

query_9 = """
SELECT title, artist
from songs
where song_id in (select song_id from listens where user_id = 1)"""

runSql('Songs listened by user 1', query_9)

title,artist
Evermore,Taylor Swift
Bad Blood,Taylor Swift
Shivers,Ed Sheeran


In [ ]:
## 10. Titles and artists of songs with >= 1 listen recorded in the Listens table.

query_10 = """
SELECT title, artist
from songs
where song_id in (select song_id from listens)
"""

runSql('Titles & Artist for listened songs', query_10)

title,artist
Evermore,Taylor Swift
Bad Blood,Taylor Swift
Shivers,Ed Sheeran
Hey Jude,Beatles
Yellow Submarine,Beatles


In [ ]:
## 11. Retrieve songs that have not been listened to by user with ID 1

query_11 = """
SELECT title, artist
from songs
where song_id not in (select song_id from listens where user_id = 1)
"""

runSql('Songs not listened by user 1', query_11)

title,artist
Willow,Taylor Swift
Shape of you,Ed Sheeran
Photograph,Ed Sheeran
Hey Jude,Beatles
Yellow Submarine,Beatles
Hey Jude,Beatles
DJ Mix,DJ


In [ ]:
## 12. Titles and artists of songs from both Pop and Rock genres

query_12 = """
SELECT title, artist
from songs
where genre = 'Pop' or genre = 'Rock'
"""

runSql('Pop & Rock <> Titles Artists-v1', query_12)

title,artist
Evermore,Taylor Swift
Bad Blood,Taylor Swift
Willow,Taylor Swift
Shape of you,Ed Sheeran
Photograph,Ed Sheeran
Shivers,Ed Sheeran


In [ ]:
## 13. Artists of songs from both Pop and Rock genres

query_13 = """
SELECT distinct artist
from songs
where genre = 'Pop' or genre = 'Rock'
"""

runSql('Pop & Rock Artists', query_13)

artist
Taylor Swift
Ed Sheeran


In [ ]:
## 14. Titles and artists of songs from both Pop and Rock genres, including duplicates

query_14 = """
SELECT title, artist
from songs
where genre = 'Pop' or genre = 'Rock'
"""

runSql('Pop & Rock <> Titles Artists-v2', query_14)

title,artist
Evermore,Taylor Swift
Bad Blood,Taylor Swift
Willow,Taylor Swift
Shape of you,Ed Sheeran
Photograph,Ed Sheeran
Shivers,Ed Sheeran


In [62]:
## 15. Artists with songs that belong to both Pop and Rock genres

query_15 = """
SELECT artist
FROM songs
WHERE genre = 'Pop' OR genre = 'Rock'
GROUP BY artist
HAVING COUNT(DISTINCT genre) = 2;
"""

runSql('Artists with Pop & Rock genres', query_15)

artist
Ed Sheeran
Taylor Swift


In [63]:
## 16. Artists with songs in Rock genre, but none in the Pop genre

query_16 = """
SELECT artist
FROM songs
WHERE genre = 'Rock'
and artist not in (Select artist from songs where genre = 'Pop');
"""

runSql('Artists with only Rock genre', query_16)

artist
